In [5]:
!git clone https://github.com/facebookresearch/segment-anything-2.git

fatal: destination path 'segment-anything-2' already exists and is not an empty directory.


In [ ]:
'https://dl.fbaipublicfiles.com/segment_anything/sam_vit_b_01ec64.pth'

In [8]:
import cv2
import torch
import numpy as np
from segment_anything import SamPredictor, sam_model_registry

In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
sam_checkpoint = "sam_vit_b_01ec64.pth"  # Provide the path to the small SAM model checkpoint
model_type = "vit_b"  # Use "vit_b" for the small version

sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
sam.to(device)
predictor = SamPredictor(sam)

# Open webcam
cap = cv2.VideoCapture(0)

In [13]:
def detect_color(mask, image):
    """Detect the average color of the masked region"""
    # Create a mask of the region
    masked_image = cv2.bitwise_and(image, image, mask=mask)
    
    # Convert to HSV color space for better color detection
    hsv = cv2.cvtColor(masked_image, cv2.COLOR_BGR2HSV)

    # Calculate the average color in the masked area
    mean_color = cv2.mean(hsv, mask=mask)
    return mean_color

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Resize the frame for faster processing
    resized_frame = cv2.resize(frame, (256, 256))
    resized_frame_rgb = cv2.cvtColor(resized_frame, cv2.COLOR_BGR2RGB)

    # Use SAM to predict the mask for the frame
    predictor.set_image(resized_frame_rgb)
    masks, scores, logits = predictor.predict(point_coords=None, point_labels=None, multimask_output=True)

    # Loop over the masks (choose the highest score mask for simplicity)
    best_mask_index = np.argmax(scores)
    best_mask = masks[best_mask_index].astype(np.uint8) * 255

    # Detect the color of the segmented object
    mean_color = detect_color(best_mask, resized_frame)

    # Display the results
    cv2.imshow("Original Frame", frame)
    cv2.imshow("Segmented Mask", best_mask)

    # Display color in HSV format
    print(f"Detected HSV Color: {mean_color}")

    # Press 'q' to quit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

Detected HSV Color: (77.27839667977283, 57.01277850589777, 64.93042813455658, 0.0)
Detected HSV Color: (42.93906588308073, 8.456387256418187, 169.53263223012684, 0.0)
Detected HSV Color: (39.94195568400771, 8.56791907514451, 170.743978805395, 0.0)
Detected HSV Color: (66.58704008786381, 6.285557386051621, 172.2720117151748, 0.0)
Detected HSV Color: (0.0, 0.0, 0.0, 0.0)
Detected HSV Color: (0.0, 0.0, 0.0, 0.0)


### for colored mask

In [ ]:
import cv2
import torch
import numpy as np
from segment_anything import SamPredictor, sam_model_registry

# Initialize the SAM model (using the small version)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
sam_checkpoint = "sam_vit_b_01ec64.pth"  # Make sure this path is correct
model_type = "vit_b"  # Use "vit_b" for the small version

sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
sam.to(device)
predictor = SamPredictor(sam)

# Open webcam
cap = cv2.VideoCapture(0)

def apply_colored_mask(image, mask, color):
    """Apply color to a binary mask and overlay on image."""
    colored_mask = np.zeros_like(image)  # Create an empty colored mask (same size as image)
    colored_mask[mask == 255] = color  # Apply the color where the mask is active
    return cv2.addWeighted(image, 1, colored_mask, 0.6, 0)  # Blend original image with the mask

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Resize the frame for faster processing
    resized_frame = cv2.resize(frame, (256, 256))
    resized_frame_rgb = cv2.cvtColor(resized_frame, cv2.COLOR_BGR2RGB)

    # Use SAM to predict the mask for the frame
    predictor.set_image(resized_frame_rgb)
    masks, scores, logits = predictor.predict(point_coords=None, point_labels=None, multimask_output=True)

    # Loop over the masks (choose the highest score mask for simplicity)
    best_mask_index = np.argmax(scores)
    best_mask = masks[best_mask_index].astype(np.uint8) * 255  # Convert to binary mask

    # Generate a random color for the mask
    color = np.random.randint(0, 255, size=(3,), dtype=int).tolist()  # Random BGR color

    # Apply the colored mask to the frame
    colored_frame = apply_colored_mask(resized_frame, best_mask, color)

    # Display the results
    cv2.imshow("Original Frame", frame)
    cv2.imshow("Colored Segmentation Mask", colored_frame)

    # Press 'q' to quit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


### colored mask to circular objects

In [ ]:
import cv2
import torch
import numpy as np
from segment_anything import SamPredictor, sam_model_registry

# Initialize the SAM model (using the small version)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
sam_checkpoint = "./sam_vit_b.pth"  # Make sure this path is correct
model_type = "vit_b"  # Use "vit_b" for the small version

sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
sam.to(device)
predictor = SamPredictor(sam)

# Open webcam
cap = cv2.VideoCapture(0)

def detect_circles(image):
    """Detect circular objects in the frame using HoughCircles"""
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blurred = cv2.medianBlur(gray, 5)
    
    # Use HoughCircles to detect circles
    circles = cv2.HoughCircles(blurred, cv2.HOUGH_GRADIENT, dp=1.2, minDist=30, param1=50, param2=30, minRadius=10, maxRadius=100)

    if circles is not None:
        circles = np.uint16(np.around(circles))
        return circles[0, :]  # Return the detected circles
    return None

def apply_colored_mask(image, mask, color):
    """Apply color to a binary mask and overlay it on the image."""
    colored_mask = np.zeros_like(image)  # Create an empty mask the same size as the image
    colored_mask[mask == 255] = color  # Apply the color where the mask is active
    return cv2.addWeighted(image, 1, colored_mask, 0.6, 0)  # Blend original image with the mask

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Resize the frame for faster processing
    resized_frame = cv2.resize(frame, (640, 480))
    resized_frame_rgb = cv2.cvtColor(resized_frame, cv2.COLOR_BGR2RGB)

    # Detect circles in the frame
    circles = detect_circles(resized_frame)

    if circles is not None:
        for circle in circles:
            x, y, r = circle
            # Create a mask for the detected circle
            mask = np.zeros_like(resized_frame[:, :, 0])  # Create a black mask
            cv2.circle(mask, (x, y), r, 255, thickness=-1)  # Fill the circle region with white (255)

            # Use SAM to predict the mask for the circular region only
            predictor.set_image(resized_frame_rgb)
            masks, scores, logits = predictor.predict(point_coords=None, point_labels=None, multimask_output=True)

            # Loop over the masks (choose the highest score mask for simplicity)
            best_mask_index = np.argmax(scores)
            best_mask = masks[best_mask_index].astype(np.uint8) * 255  # Convert to binary mask

            # Combine circle mask and segmentation mask (apply segmentation only to the circular area)
            combined_mask = cv2.bitwise_and(best_mask, mask)

            # Generate a random color for the mask
            color = np.random.randint(0, 255, size=(3,), dtype=int).tolist()  # Random BGR color

            # Apply the colored mask to the frame
            colored_frame = apply_colored_mask(resized_frame, combined_mask, color)

            # Draw the detected circle (optional for visualization)
            cv2.circle(resized_frame, (x, y), r, (0, 255, 0), 2)

            # Display the results
            cv2.imshow("Colored Segmentation Mask", colored_frame)
    else:
        cv2.imshow("Original Frame", resized_frame)

    # Press 'q' to quit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [1]:
## code with semantic segmentation
import cv2
import torch
import numpy as np
from segment_anything import SamPredictor, sam_model_registry

# Initialize the SAM model (using the small version)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
sam_checkpoint = "./sam_vit_b.pth"  # Make sure this path is correct
model_type = "vit_b"  # Use "vit_b" for the small version

sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
sam.to(device)
predictor = SamPredictor(sam)

def detect_circles(image):
    """Detect circular objects in the frame using HoughCircles"""
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blurred = cv2.medianBlur(gray, 5)
    
    # Use HoughCircles to detect circles
    circles = cv2.HoughCircles(blurred, cv2.HOUGH_GRADIENT, dp=1.2, minDist=30, param1=50, param2=30, minRadius=10, maxRadius=100)

    if circles is not None:
        circles = np.uint16(np.around(circles))
        return circles[0, :]  # Return the detected circles
    return None

def apply_colored_mask(image, mask, color):
    """Apply color to a binary mask and overlay it on the image."""
    colored_mask = np.zeros_like(image)  # Create an empty mask the same size as the image
    colored_mask[mask == 255] = color  # Apply the color where the mask is active
    return cv2.addWeighted(image, 1, colored_mask, 0.6, 0)  # Blend original image with the mask

# Load the input image from a file
image_path = "path_to_your_image.jpg"  # Replace with your image path
image = cv2.imread(image_path)

if image is None:
    print("Error: Could not load image.")
    exit()

# Resize the image for faster processing
resized_image = cv2.resize(image, (640, 480))
resized_image_rgb = cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB)

# Detect circles in the image
circles = detect_circles(resized_image)

if circles is not None:
    for circle in circles:
        x, y, r = circle
        # Create a mask for the detected circle
        mask = np.zeros_like(resized_image[:, :, 0])  # Create a black mask
        cv2.circle(mask, (x, y), r, 255, thickness=-1)  # Fill the circle region with white (255)

        # Use SAM to predict the mask for the circular region only
        predictor.set_image(resized_image_rgb)
        masks, scores, logits = predictor.predict(point_coords=None, point_labels=None, multimask_output=True)

        # Loop over the masks (choose the highest score mask for simplicity)
        best_mask_index = np.argmax(scores)
        best_mask = masks[best_mask_index].astype(np.uint8) * 255  # Convert to binary mask

        # Combine circle mask and segmentation mask (apply segmentation only to the circular area)
        combined_mask = cv2.bitwise_and(best_mask, mask)

        # Generate a random color for the mask
        color = np.random.randint(0, 255, size=(3,), dtype=int).tolist()  # Random BGR color

        # Apply the colored mask to the image
        colored_image = apply_colored_mask(resized_image, combined_mask, color)

        # Draw the detected circle (optional for visualization)
        cv2.circle(resized_image, (x, y), r, (0, 255, 0), 2)

        # Display the results
        cv2.imshow("Colored Segmentation Mask", colored_image)
else:
    cv2.imshow("Original Image", resized_image)

# Press any key to close the window
cv2.waitKey(0)
cv2.destroyAllWindows()


In [ ]:
limport cv2
import time
from segment_anything import SamPredictor, sam_model_registry

# Load the SAM model
sam = sam_model_registry["vit_h"](checkpoint="path/to/checkpoint.pth")
predictor = SamPredictor(sam)

# Open the webcam
cap = cv2.VideoCapture(0)  # Change '0' if you have multiple cameras

# Capture an image every 3 seconds and perform segmentation
while True:
    # Capture an image from the camera
    ret, frame = cap.read()
    if not ret:
        print("Failed to capture image.")
        break

    # Set the image for segmentation
    predictor.set_image(frame)
    
    # You can provide manual points for segmentation, or let the model handle it based on other criteria
    # Example point (x, y) can be provided based on prior knowledge of chip locations
    masks, scores, _ = predictor.predict(point_coords=[[x, y]], point_labels=[1])

    # Display the image with segmentation results (optional)
    binary_mask = (masks[0] > 0).astype(np.uint8)
    contours, _ = cv2.findContours(binary_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Draw contours on the frame
    for contour in contours:
        perimeter = cv2.arcLength(contour, True)
        area = cv2.contourArea(contour)
        circularity = 4 * np.pi * (area / (perimeter * perimeter))
        if 0.7 <= circularity <= 1.2:  # Circularity close to 1 indicates a circle
            cv2.drawContours(frame, [contour], -1, (0, 255, 0), 2)
    
    # Show the processed image
    cv2.imshow('Segmented Image', frame)

    # Wait for 3 seconds before capturing the next frame
    time.sleep(3)

    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the camera and close windows
cap.release()
cv2.destroyAllWindows()


In [1]:
#  LATEST SEGMENTATION CODE
# Import required libraries for both camera streaming and segmentation
from pyniryo2 import *  # For controlling Niryo robot and accessing camera
import pyniryo  # Additional functionalities from pyniryo2
import cv2 as cv  # For image display and manipulation
import torch  # For running the SAM model on GPU/CPU
import numpy as np  # For handling arrays and masks
from segment_anything import SamPredictor, sam_model_registry  # SAM model utilities for segmentation
import time  # For timing and frame rate adjustments

# Set up the ROS instance for the Niryo robot, assuming the IP address is "10.10.10.10"
ros_instance = NiryoRos("10.10.10.10")

# Initialize the vision system for the Niryo robot's camera
vision = Vision(ros_instance)

# Initialize the SAM model (using the small version)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # Use GPU if available, otherwise use CPU
sam_checkpoint = "sam_vit_b_01ec64.pth"  # Path to the pre-trained SAM model checkpoint (small version)
model_type = "vit_b"  # Using the small version of the SAM model (vit_b)

# Load the SAM model using the specified checkpoint and model type
sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
sam.to(device)  # Move the SAM model to the specified device (GPU/CPU)
predictor = SamPredictor(sam)  # Initialize the SAM predictor object

# Define a function to detect circular objects using color thresholding
def detect_circles_by_color(image):
    """Detect circular objects using color thresholding and contour analysis."""
    # Convert the image to HSV color space for better color segmentation
    hsv = cv.cvtColor(image, cv.COLOR_BGR2HSV)

    # Define the color range for detecting circles (for example, detecting red circles)
    lower_color = np.array([0, 100, 100])  # Lower bound of the color (e.g., red)
    upper_color = np.array([10, 255, 255])  # Upper bound of the color (e.g., red)

    # Create a binary mask based on the defined color range
    mask = cv.inRange(hsv, lower_color, upper_color)

    # Use morphological operations to reduce noise
    kernel = cv.getStructuringElement(cv.MORPH_ELLIPSE, (5, 5))
    mask = cv.morphologyEx(mask, cv.MORPH_CLOSE, kernel)  # Close small holes
    mask = cv.morphologyEx(mask, cv.MORPH_OPEN, kernel)   # Remove small noise

    # Find contours in the binary mask
    contours, _ = cv.findContours(mask, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

    circles = []
    for contour in contours:
        # Approximate the contour and check if it is circular
        perimeter = cv.arcLength(contour, True)
        approx = cv.approxPolyDP(contour, 0.02 * perimeter, True)

        if len(approx) >= 5:  # If the shape has enough vertices, consider it a circle
            (x, y), radius = cv.minEnclosingCircle(contour)
            circles.append((int(x), int(y), int(radius)))

    return circles

# Start the main loop to stream the Niryo camera and apply SAM segmentation
while True:
    # Step 1: Fetch the compressed image from the Niryo camera
    img_compressed = vision.get_img_compressed()

    # Step 2: Get the camera calibration details (intrinsics and distortion coefficients)
    camera_info = vision.get_camera_intrinsics()

    # Step 3: Uncompress the image to get it ready for processing
    img_uncompressed = pyniryo.uncompress_image(img_compressed)

    # Step 4: Correct the image for camera distortion using the intrinsics and distortion coefficients
    img = pyniryo.undistort_image(img_uncompressed, camera_info.intrinsics, camera_info.distortion)

    # Step 5: Resize the frame for processing (optional)
    resized_frame = cv.resize(img, (640, 480))

    # Step 6: Detect circles in the resized frame using color thresholding
    circles = detect_circles_by_color(resized_frame)

    # Step 7: Prepare to store masks for segmented circles
    if circles:
        for circle in circles:
            x, y, r = circle  # Get the circle's center (x, y) and radius (r)

            # Create a mask for the detected circle
            circle_mask = np.zeros((resized_frame.shape[0], resized_frame.shape[1]), dtype=np.uint8)  # Create a black mask
            cv.circle(circle_mask, (x, y), r, 255, thickness=-1)  # Fill the circle region with white (255)

            # Convert the resized frame to RGB format for SAM
            resized_frame_rgb = cv.cvtColor(resized_frame, cv.COLOR_BGR2RGB)

            # Set the image for segmentation using SAM
            predictor.set_image(resized_frame_rgb)

            # Perform semantic segmentation on the entire frame
            masks, scores, _ = predictor.predict(point_coords=None, point_labels=None)

            # Use the best mask (highest score)
            best_mask_index = np.argmax(scores)
            best_mask = masks[best_mask_index].astype(np.uint8) * 255  # Convert the SAM mask to a binary mask (0 or 255)

            # Combine the circle mask and the segmentation mask
            combined_mask = cv.bitwise_and(best_mask, circle_mask)

            # Draw the contours of the combined mask on the original image
            contours, _ = cv.findContours(combined_mask, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
            for contour in contours:
                cv.drawContours(resized_frame, [contour], -1, (0, 255, 0), 2)  # Draw contours in green

    # Display the results
    cv.imshow("Segmented Circles", resized_frame)

    # Press 'q' to quit the loop
    if cv.waitKey(1) & 0xFF == ord('q'):
        break

# Cleanup resources
cv.destroyAllWindows()  # Close all OpenCV windows

ModuleNotFoundError: No module named 'pyniryo2'